In [1]:
%load_ext autoreload
%load_ext watermark
%autoreload 2
%watermark -v -n -m -p numpy,scipy,sklearn,pandas,tensorflow,keras

Wed Feb 20 2019 

CPython 3.6.8
IPython 7.2.0

numpy 1.15.4
scipy 1.2.0
sklearn not installed
pandas 0.23.4
tensorflow 1.12.0
keras 2.2.4

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-43-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


Using TensorFlow backend.


In [2]:
# TODO visualise and summarise the corpus data

In [81]:
from data.preprocess import PreProcessor

data = PreProcessor(config=PreProcessor.DEFAULT_CONFIG,
                    data_dir='data/raw/r252-corpus-features/org/elasticsearch/action/admin/cluster/allocation/')

vocab = data.metadata['token_vocab']
processed = data.get_tensorise_data()
code_snippet = processed['body_tokens']
label_name = processed['name_tokens']

In [98]:
from tensorflow.python import keras

from models.cnn_attention import ConvAttention

embedding_dim = 128
vocabulary_size = len(vocab)
max_chunk_length_ = data.config['max_chunk_length']

# Optimised hyperparameter are reported in page 5 of the paper
model = ConvAttention(vocabulary_size,
                      embedding_dim,
                      max_chunk_length_,
                      k1=8,
                      k2=8,
                      w1=24,
                      w2=29,
                      w3=10,
                      dropout_rate=0.5,
                      do_dropout=True)
optimizer = keras.optimizers.Nadam()  # RMSprop with Nesterov momentum
loss_func = keras.losses.categorical_crossentropy

model.compile(optimizer=optimizer,
              loss=loss_func,
              metrics=['accuracy'])
# fit the model
model.fit(code_snippet, label_name, epochs=50, verbose=0)
model.summary()


ValueError: Input 0 of layer permute_2 is incompatible with the layer: expected ndim=4, found ndim=3. Full shape received: [None, 50, 128]

In [23]:

# overfit and evaluate the model 
loss, accuracy = model.evaluate(code_snippet, label_name, verbose=0)
print('Accuracy: {}'.format(accuracy * 100))


ValueError: Error when checking target: expected conv1d_5 to have 3 dimensions, but got array with shape (64, 50)

In [83]:
from data.utils import translate_tokenized_array_to_list_words

translate_tokenized_array_to_list_words(vocab, label_name[4])

['test', 'allocation', 'filtering', '%UNK%', 'shard', 'move']